In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930

year = 2021
path_prefix = f"{year}/"

In [ ]:
# load data for each generator
pudl_out = load_data.initialize_pudl_out(year)
gens = pudl_out.gens_eia860()

# load data about the balancing authority of each plant
plant_ba = pudl_out.plants_eia860()[
    ["plant_id_eia", "balancing_authority_code_eia"]
].rename(columns={"balancing_authority_code_eia": "ba_code"})

# merge the ba code into the gens data
gens = gens.merge(plant_ba, how="left", on="plant_id_eia", validate="m:1")

# load wind data
wind = pd.read_excel(
    downloads_folder(f"eia860/eia860{year}/3_2_Wind_Y{year}.xlsx"),
    sheet_name="Operable",
    header=1,
).rename(columns=
    {
        "Plant Code": "plant_id_eia",
        "Generator ID": "generator_id",
        "Design Wind Speed (mph)": "rated_speed_mph",
        "Turbine Hub Height (Feet)": "hub_height_ft",
    }
)

# convert to metric units
wind["rated_speed_m_per_s"] = (wind["rated_speed_mph"] * 0.44704).round(1)
wind["hub_height_m"] = (wind["hub_height_ft"] * 0.3048).round(0)

wind = wind[["plant_id_eia", "generator_id", "rated_speed_m_per_s", "hub_height_m"]]

# merge the wind data into the gens data
gens = gens.merge(wind, how="left", on=["plant_id_eia","generator_id"])

gens.to_csv(outputs_folder(f"gens_{year}.csv"), index=False)